# Creating an Agent using Large Language Models (LLMs)

**Goal:** Create a natural language interface for a simple API that reads / write to a redis cache as a proof of concept. 

Examples: 
1. Create a new record for a new user Ryan Skinner, locaed in Denver Colorado
2. Retreieve a record for the user Ryan Skinner

### Basic Configuration of the OpenAI Client

In [1]:
import os
import ssl
import json

from openai import OpenAI
from ddtrace.llmobs import LLMObs

In [2]:
# Load the secrets
with open("secrets.json") as f:
    secrets = json.load(f)
DD_API_KEY = secrets["DD_API_KEY"]
DD_SITE = secrets["DD_SITE"]
OAI_API_KEY = secrets["OAI_API_KEY"]

# Enable the integration
LLMObs.enable(
    integrations_enabled=True, 
    ml_app="skinner-OAi-llm-agent", 
    api_key = DD_API_KEY,
    site = DD_SITE,
    agentless_enabled = True,
    env="test",
    service="llm-agent"
)

# Establish the Client
client = OpenAI(api_key=OAI_API_KEY)


In [3]:
# Testing if the Client Works
completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with"
                                  " creative flair."},
    {"role": "user", "content": "Can you write a haiku about kubernetes"}
  ]
)

In [4]:
print(completion.choices[0].message)

ChatCompletionMessage(content="Clusters dance in sync,  \nPods afloat on orchestration,  \nCloud's dream takes its flight.  ", refusal=None, role='assistant', function_call=None, tool_calls=None)


### Integrating with LangChain

In [5]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=OAI_API_KEY)
llm.invoke("What is 2+2?")

AIMessage(content='2 + 2 = 4', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0d6ac765-a803-4840-ae20-1d7b48f481c0-0', usage_metadata={'input_tokens': 14, 'output_tokens': 7, 'total_tokens': 21})

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([ 
    ("system","You are chatGPT"), 
    ("user","{input}")
])
chain = prompt | llm
chain.invoke("What is 2 + 2?")

AIMessage(content='2 + 2 equals 4.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 24, 'total_tokens': 32}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-10bd2d9f-4414-4156-bc1b-656278c4c420-0', usage_metadata={'input_tokens': 24, 'output_tokens': 8, 'total_tokens': 32})

## Prompt engineering to classify a users text

In [34]:
system_prompt = "You are responsible for classifying a users request as the following \n\
    1. A request to add a user to the database \n\
    2. Update the user information to the database\n\
    3. Retrieve information about the user from the database\n\n\
The only valid responses are ADD, UPDATE, RETRIEVE, UNKNOWN\n\n\
If the user intends to add a user respond with ADD\n\
If the user intends to update user information in the database respond with UPDATE\n\
If the user inteds to retrieve information from the database respond with RETRIEVE\n\
If unknown or uncertain respond with UNKNOWN"
print(system_prompt)

You are responsible for classifying a users request as the following 
    1. A request to add a user to the database 
    2. Update the user information to the database
    3. Retrieve information about the user from the database

The only valid responses are ADD, UPDATE, RETRIEVE, UNKNOWN

If the user intends to add a user respond with ADD
If the user intends to update user information in the database respond with UPDATE
If the user inteds to retrieve information from the database respond with RETRIEVE
If unknown or uncertain respond with UNKNOWN


In [35]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user","{input}")
])

In [40]:
chain = prompt | llm
chain.invoke("Where does Ryan live?")

AIMessage(content='RETRIEVE', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 136, 'total_tokens': 140}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-69916451-b2b5-4279-97ef-7433d83a3a15-0', usage_metadata={'input_tokens': 136, 'output_tokens': 4, 'total_tokens': 140})